In [120]:
from libpy import readfile

#filename = "2024/day06/input/sample.txt"
filename = "2024/day06/input/input.txt"
lines = readfile.lines(filename)

height = len(lines)
width = len(lines[0])
blocks = []
start = None
for y in range(height):
    for x in range(width):
        if lines[y][x] == "^":
            start = (x, y)
        elif lines[y][x] == "#":
            blocks.append((x, y))
            
print(str(start[0]) + "," + str(start[1]))
print(blocks)

78,61
[(30, 0), (64, 0), (80, 0), (89, 0), (91, 0), (94, 0), (101, 0), (104, 0), (113, 0), (118, 0), (18, 1), (19, 1), (29, 1), (60, 1), (72, 1), (95, 1), (104, 1), (15, 2), (31, 2), (63, 2), (67, 2), (98, 2), (104, 2), (120, 2), (125, 2), (128, 2), (25, 3), (83, 3), (108, 3), (0, 4), (4, 4), (6, 4), (16, 4), (19, 4), (46, 4), (75, 4), (97, 4), (54, 5), (77, 5), (108, 5), (119, 5), (129, 5), (38, 6), (40, 6), (46, 6), (67, 6), (102, 6), (103, 6), (15, 7), (37, 7), (48, 7), (72, 7), (90, 7), (103, 7), (104, 7), (127, 7), (109, 8), (9, 9), (11, 9), (22, 9), (27, 9), (34, 9), (52, 9), (75, 9), (74, 10), (107, 10), (110, 10), (117, 10), (13, 11), (26, 11), (39, 11), (74, 11), (87, 11), (99, 11), (106, 11), (123, 11), (20, 12), (29, 12), (30, 12), (83, 12), (125, 12), (74, 13), (80, 13), (94, 13), (127, 13), (3, 14), (5, 14), (11, 14), (23, 14), (24, 14), (27, 14), (51, 14), (113, 14), (14, 15), (24, 15), (34, 15), (62, 15), (98, 15), (110, 15), (128, 15), (129, 15), (20, 16), (50, 16), (58

In [127]:
from collections import defaultdict

up = (0, -1)
right = (1, 0)
bottom = (0, 1)
left = (-1, 0)

def turn(direction):
    if direction == up:
        return right
    elif direction == right:
        return bottom
    elif direction == bottom:
        return left
    elif direction == left:
        return up
        
def find_blocks(blocks, x1, y1, x2, y2):
    return [b for b in blocks if b[0] >= min(x1,x2) and b[0] <= max(x1,x2) and b[1] >= min(y1,y2) and b[1] <= max(y1,y2)]

def find_blocks2(blocks_per_x, blocks_per_y, x1, y1, x2, y2):
    if x1 == x2:
        return [b for b in blocks_per_x[x1] if b[1] >= min(y1,y2) and b[1] <= max(y1,y2)]
    if y1 == y2:
        return [b for b in blocks_per_y[y1] if b[0] >= min(x1,x2) and b[0] <= max(x1,x2)]

def get_end(position, direction):
    if direction == up:
            return (position[0], 0)
    elif direction == right:
        return (width - 1, position[1])
    elif direction == bottom:
        return (position[0], height - 1)
    elif direction == left:
        return (0, position[1])

def is_edge(position):
    return position[0] == 0 or position[0] == width - 1 or position[1] == 0 or position[1] == height - 1

def find_route(blocks, position, direction):
    visited = set()
    visited_per_direction = {up: set(), right: set(), bottom: set(), left: set()}
    while not is_edge(position):
        end = get_end(position, direction)
        blocked_by = find_blocks(blocks, position[0], position[1], end[0], end[1])
        if len(blocked_by) == 0:
            new_position = end
        else:
            first_block = sorted(blocked_by, key=lambda b: b[abs(direction[1])], reverse=direction[0] < 0 or direction[1] < 0)[0]
            new_position = (first_block[0] - direction[0], first_block[1] - direction[1])
        
        visits = []
        for x in range(min(position[0], new_position[0]), max(position[0], new_position[0]) + 1):
            for y in range(min(position[1], new_position[1]), max(position[1], new_position[1]) + 1):
                visits.append((x,y))
        
        visited_len = len(visited_per_direction[direction])
        visited_per_direction[direction].update(visits)
        if visited_len == len(visited_per_direction[direction]):
            return None
        
        visited.update(visits)
        
        direction = turn(direction)    
        position = new_position
    
    return visited

def is_loop(blocks, position, direction):
    visited_per_direction = {up: set(), right: set(), bottom: set(), left: set()}
    blocks_per_x = defaultdict(list)
    blocks_per_y = defaultdict(list)
    for b in blocks:
        blocks_per_x[b[0]].append(b)
        blocks_per_y[b[1]].append(b)
    
    while not is_edge(position):
        if position in visited_per_direction[direction]:
            return True
        visited_per_direction[direction].add(position)
        
        end = get_end(position, direction)
        blocked_by = find_blocks2(blocks_per_x, blocks_per_y, position[0], position[1], end[0], end[1])
        if len(blocked_by) == 0:
            new_position = end
        else:
            first_block = sorted(blocked_by, key=lambda b: b[abs(direction[1])], reverse=direction[0] < 0 or direction[1] < 0)[0]
            new_position = (first_block[0] - direction[0], first_block[1] - direction[1])
        
        direction = turn(direction)    
        position = new_position
    
    return False

Part A

In [131]:
route = find_route(blocks, start, up)
if route is not None:
    print(len(route))

5331


Part B

In [129]:
score = 0
route = find_route(blocks, start, up)

for b in route:
    if is_loop(blocks + [b], start, up):
        score += 1
            
print(score)

1812
